# Modello LSMT (Long Short-Term Memory) 

### Codice di Configurazione e Pre-Elaborazione per LSTM
Questo blocco di codice svolge due funzioni principali: prepara l'ambiente di calcolo, ottimizzando l'uso della GPU con TensorFlow, e definisce i parametri cruciali per la lettura dei dati e la costruzione del modello LSTM (Long Short-Term Memory) per la previsione 
delle traiettorie.


In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import joblib
import pyarrow.parquet as pq
from collections import Counter
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras import mixed_precision # Per la 4050
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, TimeDistributed, RepeatVector
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU Rilevata: {len(gpus)} (Memory Growth Attiva)")
        
        policy = mixed_precision.Policy('mixed_float16')
        mixed_precision.set_global_policy(policy)
        print("Mixed Precision (FP16) Attivata")
        
    except RuntimeError as e:
        print(e)
else:
    print("ATTENZIONE:CPU.")

INPUT_DIR = '../Pre-Elaborazione Dati/Dataset' 
SCALER_PATH = 'scaler.joblib' 
COLONNE_FEATURES = ['Latitude', 'Longitude', 'SOG', 'COG']

WINDOW_SIZE = 30  
BATCH_SIZE = 64

all_files = sorted(glob.glob(os.path.join(INPUT_DIR, '*.parquet')))
TRAIN_FILES = all_files[0:16]
VAL_FILES = all_files[16:20]
TEST_FILES = all_files[20:24] 

print("Configurazione completata.")

2025-11-20 12:51:06.012008: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-20 12:51:06.049014: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-20 12:51:06.877732: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


GPU Rilevata: 1 (Memory Growth Attiva)
Mixed Precision (FP16) Attivata
Configurazione completata.


### Pre-screening delle Traiettorie di Training e Valutazione del Timestep (Window Size)
Questo blocco di codice svolge una funzione di analisi preliminare critica dei dati di training. Il suo scopo principale è determinare l'usabilità delle traiettorie in funzione della dimensione della finestra temporale (Window Size o Timestep), un iperparametro cruciale per il modello LSTM.
Il codice segue tre fasi distinte:
1. **Caricamento e Preparazione dei File di Training**:
   il codice individua tutti i file di dati che contengono le traiettorie delle navi (i file Parquet) e ne seleziona un sottoinsieme specifico da utilizzare per il training. Quindi, definisce una lista di diverse lunghezze di finestra che vogliamo testare (ad esempio, se usiamo gli ultimi 30 punti per la previsione o gli ultimi 40).
   
2. **Misurazione delle Traiettorie**: il codice scorre l'intero set di training, identificando ogni singola traiettoria unica e,     soprattutto, contando esattamente quanti punti (o misure) essa contiene. In pratica, misura la lunghezza di ogni sequenza di navigazione.

3. **Report di Usabilità**: il codice combina le lunghezze misurate con i timestep che vogliamo testare. Il risultato è un report statistico:
   * Per ogni dimensione di finestra (ad esempio, 30 punti), il codice calcola quante traiettorie sono abbastanza lunghe (almeno 30 punti) e quindi utilizzabili per l'addestramento.

   * Conseguentemente, calcola quante traiettorie sono troppo corte e devono essere scartate per quella specifica configurazione.
 


In [ ]:
all_files = sorted(glob.glob(os.path.join(INPUT_DIR, '*.parquet')))

if len(all_files) == 0:
    print(f"ERRORE CRITICO: Nessun file .parquet trovato in '{INPUT_DIR}'")
else:
    print(f"Trovati {len(all_files)} file .parquet in '{INPUT_DIR}'")

TRAIN_FILES = all_files[0:16]
IPERPARAMETRI_TIMESTEP = [10, 15, 20, 25, 30, 40]

print(f"Controllo iperparametri sui file di TRAINING")

total_counts_counter = Counter()
total_ids_set = set()

for file_path in TRAIN_FILES:
    print(f"  Processando {os.path.basename(file_path)}")
    try:
        df = pd.read_parquet(file_path, columns=['TrajectoryID'])
        
        total_ids_set.update(df['TrajectoryID'].unique())
        
        total_counts_counter.update(df['TrajectoryID'])
        
        del df
        gc.collect()
    except Exception as e:
        print(f"ERRORE lettura:{file_path}: {e}")
        

conteggio_set = len(total_ids_set)
conteggio_counter = len(total_counts_counter)

if conteggio_set != conteggio_counter:
    print("Errore:I conteggi non corrispondono. C'è ancora un problema.")
else:
    print("\nI conteggi corrispondono.")
    
    all_lengths = list(total_counts_counter.values())
    totale_traiettorie_analisi = conteggio_counter

    print(f"\nRisultati dell'analisi (su {totale_traiettorie_analisi:,} traiettorie totali di TRAINING):")
    print("-" * 75)
    print(f"{'Window Size':<15} | {'Traiettorie Usabili':<20} | {'Traiettorie Scartate':<20} | {'% Usabili':<10}")
    print("-" * 75)

    for timestep in IPERPARAMETRI_TIMESTEP:
        traiettorie_usabili = sum(1 for length in all_lengths if length >= timestep)
        traiettorie_scartate = totale_traiettorie_analisi - traiettorie_usabili
        percentuale_usabili = (traiettorie_usabili / totale_traiettorie_analisi) * 100
        print(f"{timestep:<15} | {traiettorie_usabili:<20} | {traiettorie_scartate:<20} | {percentuale_usabili:<10.2f}%")

    print("-" * 75)
    print("Analisi completata.")

Trovati 25 file .parquet in '../Pre-Elaborazione Dati/Dataset'
Controllo iperparametri sui file di TRAINING
  Processando blocco_000-segmentato.parquet
  Processando blocco_001-segmentato.parquet
  Processando blocco_002-segmentato.parquet
  Processando blocco_003-segmentato.parquet
  Processando blocco_004-segmentato.parquet
  Processando blocco_005-segmentato.parquet
  Processando blocco_006-segmentato.parquet
  Processando blocco_007-segmentato.parquet
  Processando blocco_008-segmentato.parquet
  Processando blocco_009-segmentato.parquet
  Processando blocco_010-segmentato.parquet
  Processando blocco_011-segmentato.parquet
  Processando blocco_012-segmentato.parquet
  Processando blocco_013-segmentato.parquet
  Processando blocco_014-segmentato.parquet
  Processando blocco_015-segmentato.parquet

I conteggi corrispondono.

Risultati dell'analisi (su 4,369,967 traiettorie totali di TRAINING):
---------------------------------------------------------------------------
Window Size   

### Normalizzazione

Passaggio fondamentale perchè le reti neurali (LSTM/LNN) faticano a imparare quando i dati di input hanno scale completamente diverse. Ad esempio, la colonna COG (rotta) va da 0 a 360, mentre la colonna SOG (velocità) potrebbe andare da 0 a 50. Questi intervalli così diversi "confondono" la rete durante l'addestramento.
Dobbiamo quindi standardizzarli, portando tutte le colonne a una scala simile.  
Il modo corretto per standardizzare è calcolare la media e la deviazione standard dell'intero set di addestramento (tutti i 4.3 milioni di traiettorie nei tuoi 16 file). Ovviamente, questi dati sono troppo grandi per essere caricati tutti insieme in memoria.

Quello che facciamo è:  
- Inizializziamo uno scaler vuoto e iteriamo sui 16 file di Train.
- `partial_fit` è la parte fondamentale perchè per ogni file che carica. Questa funzione dice allo scaler: "Prendi la media e la deviazione che hai calcolato finora e aggiornale includendo questo nuovo blocco di dati".
- `joblib.dump(scaler, SCALER_PATH)` salva questi parametri calcolati in un file esterno.


In [5]:
COLONNE_DA_NORMALIZZARE = ['Latitude', 'Longitude', 'SOG', 'COG']
SCALER_PATH = 'scaler.joblib'

scaler = StandardScaler()

try:
    
    for i, file_path in enumerate(TRAIN_FILES):
        
        print(f"Processando file {i+1}/{len(TRAIN_FILES)}: {os.path.basename(file_path)}")
        
        df_chunk = pd.read_parquet(file_path, columns=COLONNE_DA_NORMALIZZARE)
        
        scaler.partial_fit(df_chunk)
        
        del df_chunk
        gc.collect()

    print("\nAdattamento completato su tutti i 16 file di training.")
    
    joblib.dump(scaler, SCALER_PATH) #Salva file

    print(f"\nScaler salvato come '{SCALER_PATH}'.")

except Exception as e:
    print(f"\n Errore durante la preparazione dello scaler: {e}")

Processando file 1/16: blocco_000-segmentato.parquet
Processando file 2/16: blocco_001-segmentato.parquet
Processando file 3/16: blocco_002-segmentato.parquet
Processando file 4/16: blocco_003-segmentato.parquet
Processando file 5/16: blocco_004-segmentato.parquet
Processando file 6/16: blocco_005-segmentato.parquet
Processando file 7/16: blocco_006-segmentato.parquet
Processando file 8/16: blocco_007-segmentato.parquet
Processando file 9/16: blocco_008-segmentato.parquet
Processando file 10/16: blocco_009-segmentato.parquet
Processando file 11/16: blocco_010-segmentato.parquet
Processando file 12/16: blocco_011-segmentato.parquet
Processando file 13/16: blocco_012-segmentato.parquet
Processando file 14/16: blocco_013-segmentato.parquet
Processando file 15/16: blocco_014-segmentato.parquet
Processando file 16/16: blocco_015-segmentato.parquet

Adattamento completato su tutti i 16 file di training.

Scaler salvato come 'scaler.joblib'.


#### Funzioni del Data Generator
Questo insieme di funzioni costituisce il motore di preparazione dei dati per l'addestramento della rete neurale LSTM, risolvendo il problema di addestrare un modello su milioni di punti di dati (traiettorie) con una memoria di sistema limitata.

1. **Funzione di Calcolo Passi per Epoca** *(calculate_steps_per_epoch)*
Quando si addestra una rete neurale, è necessario sapere esattamente quanti step (ovvero, quanti batch di dati) compongono un'epoca completa. Questa funzione svolge questo compito in modo sicuro e robusto:

   * **Conteggio Sicuro:** La funzione scansiona tutti i file di dati leggendo solo l'ID di ogni traiettoria e contandone la lunghezza, esattamente come nell'analisi precedente. Questo è un processo efficiente in termini di RAM perché non carica tutte le colonne in memoria.

   * **Calcolo delle Finestre:** Per ogni traiettoria la cui lunghezza supera la WINDOW_SIZE (la lunghezza di input richiesta dall'LSTM), calcola il numero esatto di finestre temporali che possono essere estratte (una traiettoria lunga 100 punti può generare 100 - WINDOW_SIZE + 1 finestre).

   * **Determinazione degli Steps:** Infine, divide il numero totale di finestre (campioni) per la BATCH_SIZE (il numero di campioni elaborati per volta dalla GPU). Il risultato è il numero preciso di steps per epoca che la funzione di training di Keras dovrà eseguire.

   Questa precisione è cruciale: se il numero di steps è errato, l'addestramento si blocca o non copre l'intero dataset, compromettendo l'apprendimento.

2. **Generatore di Dati Efficace** *(data_generator_v)*
Poiché il dataset delle traiettorie è troppo grande per stare nella RAM, non possiamo usare i metodi di caricamento standard. È necessario un Data Generator che agisca come una *fabbrica di dati just-in-time*

   * **Lettura Chunkizzata e Sequenziale:** Il generatore legge i file di dati Parquet in piccoli blocchi (CHUNK_SIZE_ROWS) anziché tutto in una volta. Questo assicura che il carico di memoria rimanga basso. 

   * **Gestione delle Traiettorie a Cavallo:** La sfida principale è che una singola traiettoria può estendersi su più blocchi di dati (chunk) o addirittura su più file. Il generatore utilizza dei buffer temporanei (chunk_buffer e file_buffer) per memorizzare i dati parziali di una traiettoria finché non è completa, garantendo che nessuna sequenza venga interrotta o persa.

   * **Creazione delle Finestre:** Per ogni traiettoria completa, la funzione interna *create_windows* estrae tutte le finestre temporali di dimensione WINDOW_SIZE che possono essere generate.

   * **Normalizzazione e Yield:**

     1. I dati estratti vengono immediatamente normalizzati usando lo scaler pre-calcolato (portandoli sulla scala media 0, deviazione standard 1).

     2. Quando il generatore ha abbastanza finestre per riempire un BATCH_SIZE, le impacchetta e le fornisce alla rete neurale tramite l'istruzione yield.


In [3]:
def calculate_steps_per_epoch(file_paths, window_size, batch_size):
    """
    Scansiona tutti i file in modo efficiente (RAM-safe) per calcolare
    il numero totale di finestre (campioni) che verranno generate,
    e da lì calcola il numero di "steps" (batch) per epoca.
    Usa il metodo Counter.update(Series) che sappiamo funzionare.
    """
    print(f"--- Calcolo Steps per {len(file_paths)} file ---")
    
    total_lengths = Counter()
    
    #Trova le lunghezze di tutte le 4.3M traiettorie
    for i, file_path in enumerate(file_paths):
        print(f"  Scansione lunghezze file {i+1}/{len(file_paths)}...", end='\r')
        try:
            df = pd.read_parquet(file_path, columns=['TrajectoryID'])
            total_lengths.update(df['TrajectoryID'])
            del df
            gc.collect()
        except Exception as e:
            print(f"Errore nel leggere {file_path}: {e}")

    print("\nScansione completata.")

    #Calcola il numero totale di finestre
    total_windows = 0
    for length in total_lengths.values():
        if length >= window_size:
            total_windows += (length - window_size + 1)
            
    #Calcola gli steps
    steps = int(np.ceil(total_windows / batch_size))
    
    print(f"Trovate {total_windows:,} finestre totali.")
    print(f"Steps per Epoca (Batch Size {batch_size}): {steps}")
    print("-" * 75)
    return steps


def create_windows(data_np, window_size):
    windows = []
    for i in range(len(data_np) - window_size + 1):
        windows.append(data_np[i : i + window_size])
    return windows

def data_generator_v(file_paths, scaler, features, window_size, batch_size, shuffle_files=False):
    
    file_buffer = {} 
    window_buffer = [] 
    CHUNK_SIZE_ROWS = 500_000

    while True:
        if shuffle_files:
             # Shuffle disattivato forzatamente per garantire la sequenzialità
            shuffle_files = False 
            
        for file_path in file_paths:
            chunk_buffer = {}
            try:
                pf = pq.ParquetFile(file_path)
                for batch in pf.iter_batches(batch_size=CHUNK_SIZE_ROWS, columns=features + ['TrajectoryID']):
                    df_chunk = batch.to_pandas()
                    df_chunk[features] = scaler.transform(df_chunk[features])
                    next_chunk_buffer = {}
                    
                    for tid, group in df_chunk.groupby('TrajectoryID'):
                        if tid in chunk_buffer:
                            trajectory_data = pd.concat([chunk_buffer.pop(tid), group])
                        else:
                            trajectory_data = group
                        
                        if tid in file_buffer:
                            trajectory_data = pd.concat([file_buffer.pop(tid), trajectory_data])
                        
                        # Se la traiettoria tocca la fine del chunk, bufferizzala
                        if trajectory_data.iloc[-1].name == df_chunk.iloc[-1].name:
                            next_chunk_buffer[tid] = trajectory_data
                            continue 
                            
                        if len(trajectory_data) < window_size:
                            continue 
                            
                        trajectory_np = trajectory_data[features].to_numpy()
                        new_windows = create_windows(trajectory_np, window_size)
                        window_buffer.extend(new_windows)
                        
                        next_chunk_buffer[tid] = trajectory_data.iloc[-(window_size - 1):]

                        while len(window_buffer) >= batch_size:
                            batch_to_yield = window_buffer[:batch_size]
                            window_buffer = window_buffer[batch_size:]
                            yield (np.array(batch_to_yield), np.array(batch_to_yield))
                    
                    chunk_buffer = next_chunk_buffer
                file_buffer = chunk_buffer
            except Exception as e:
                print(f"\nErrore lettura {file_path}: {e}")
                continue
print("Funzioni definite")

Funzioni definite


#### Calcolo degli steps

In [3]:
print("Avvio calcolo steps per il Training Set...")
train_steps = calculate_steps_per_epoch(TRAIN_FILES, WINDOW_SIZE, BATCH_SIZE)

print("\nAvvio calcolo steps per il Validation Set...")
val_steps = calculate_steps_per_epoch(VAL_FILES, WINDOW_SIZE, BATCH_SIZE)

print("\n--- Calcolo Steps Completato ---")
print(f"Training Steps per Epoca: {train_steps}")
print(f"Validation Steps per Epoca: {val_steps}")

Avvio calcolo steps per il Training Set...
--- Calcolo Steps per 16 file ---
  Scansione lunghezze file 16/16...
Scansione lunghezze completata.
Trovate 404,801,806 finestre totali.
Steps per Epoca (Batch Size 64): 6325029
------------------------------------------

Avvio calcolo steps per il Validation Set...
--- Calcolo Steps per 4 file ---
  Scansione lunghezze file 4/4...
Scansione lunghezze completata.
Trovate 90,369,948 finestre totali.
Steps per Epoca (Batch Size 64): 1412031
------------------------------------------

--- Calcolo Steps Completato ---
Training Steps per Epoca: 6325029
Validation Steps per Epoca: 1412031


#### Caricamento Scaler e creazione dei Generatori

In [4]:
print("Inizializzazione generatori")
scaler = joblib.load(SCALER_PATH)

train_gen = data_generator_v(
    file_paths=TRAIN_FILES,
    scaler=scaler,
    features=COLONNE_FEATURES,
    window_size=WINDOW_SIZE,
    batch_size=BATCH_SIZE,
    shuffle_files=False 
)

val_gen = data_generator_v(
    file_paths=VAL_FILES,
    scaler=scaler,
    features=COLONNE_FEATURES,
    window_size=WINDOW_SIZE,
    batch_size=BATCH_SIZE,
    shuffle_files=False
)
print("Generatori pronti.")

Inizializzazione generatori
Generatori pronti.


#### Definizione Modello LSMT Autoencoder
Questa parte del codice descrive la costruzione della rete neurale vera e propria che sarà utilizzata per apprendere e, successivamente, rilevare le anomalie nelle traiettorie delle navi. Il modello implementato è un Autoencoder Sequenziale basato su unità LSTM (Long Short-Term Memory).
L'obiettivo di un Autoencoder è imparare una rappresentazione compressa dei dati (il vettore latente) e poi ricostruire l'input originale dall'output.

1. La Fase di Codifica **(Encoder)**

   * **Input:** La rete accetta come input una finestra temporale (WINDOW_SIZE) con le nostre caratteristiche (**n_features**: Latitudine, Longitudine, SOG, COG).

   * **LSTM Encoder:** Lo strato LSTM prende la sequenza completa e la comprime. L'impostazione **return_sequences=False** è cruciale: significa che l'LSTM non restituisce una sequenza completa, ma solo l'ultimo stato nascosto. Questo stato è il vettore latente (latent_dim = 32), che rappresenta la versione compressa e concettuale dell'intera traiettoria di input.

2. La Fase di Decodifica **(Decoder)**

   * **Repeat Vector:** Il vettore latente (la rappresentazione compressa) viene replicato lungo l'asse temporale per la lunghezza originale della finestra (WINDOW_SIZE). Questo "spalma" l'informazione compressa per prepararla alla decodifica.

   * **LSTM Decoder:** Un secondo strato LSTM riceve il vettore replicato. Questa volta, **return_sequences=True** indica che deve generare una sequenza completa. Il suo compito è "srotolare" il vettore latente e ricostruire la sequenza temporale di output punto per punto.

   * **TimeDistributed Dense:** Il layer finale *Dense* è applicato in modo indipendente a ogni singolo punto temporale dell'output del decoder. Questo rimappa i risultati interni dell'LSTM ricostruendo le quattro caratteristiche originali **(n_features)**.

3. **Compilazione del Modello**

   * **Definizione del Modello:** *Model(inputs, output)* unisce l'encoder e il decoder in un'unica architettura end-to-end.

   * **Ottimizzatore (*adam*):** L'algoritmo *Adam* è scelto per l'ottimizzazione del peso del modello, noto per la sua efficacia e velocità.

   * **Funzione di Loss** *(loss='mae')*: Viene utilizzata la **Mean Absolute Error (MAE)**. Questa funzione calcola la differenza media assoluta tra la sequenza di traiettoria di input e la sequenza ricostruita in output. Il modello cercherà di minimizzare questo **MAE** durante l'addestramento, imparando a ricostruire le sequenze normali con l'errore più basso possibile.
  



In [5]:

n_features = len(COLONNE_FEATURES)
latent_dim = 32

# Encoder
inputs = Input(shape=(WINDOW_SIZE, n_features))
# Il primo LSTM comprime l'input
lstm_encoder = LSTM(latent_dim, return_sequences=False)(inputs)

# Decoder
# Ripete il vettore compresso per ogni timestep
repeat_vector = RepeatVector(WINDOW_SIZE)(lstm_encoder)

# Il secondo LSTM "legge" il vettore compresso e ricostruisce la sequenza
lstm_decoder = LSTM(latent_dim, return_sequences=True)(repeat_vector)

# Un layer finale per rimappare l'output
output = TimeDistributed(Dense(n_features))(lstm_decoder)

model_lstm = Model(inputs, output)
model_lstm.compile(optimizer='adam', loss='mae')

print("Modello LSTM-Autoencoder creato e compilato.")
model_lstm.summary()

I0000 00:00:1763639954.128417   13211 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4130 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Modello LSTM-Autoencoder creato e compilato.


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 30, 4)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 32)             │         4,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ repeat_vector (RepeatVector)    │ (None, 30, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 30, 32)         │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 30, 4)          │           132 │
│ (TimeDistributed)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13,188 (51.52 KB)

 Trainable params: 13,188 (51.52 KB)

 Non-trainable params: 0 (0.00 B)

#### Addestramento LSMT
1. **Definizione dei Parametri di Training Ottimali**
Prima di iniziare, vengono stabiliti i parametri di dimensione dell'epoca basati su test preliminari eseguiti sulla GPU RTX 4050:

   * **STEPS_PER_EPOCH_CALCOLATI (200.000):** Questo valore, ottenuto da analisi di benchmark sulla GPU (come la funzione calculate_steps_per_epoch discussa precedentemente), definisce la quantità di dati che il modello deve vedere per completare un'epoca. Questo numero è stato specificamente calibrato per far sì che ogni epoca duri circa 30 minuti, ottimizzando l'uso delle risorse GPU.

   * **VALIDATION_STEPS_CALCOLATI (40.000):** Definisce quanti batch di dati di validazione verranno utilizzati al termine di ogni epoca per valutare le prestazioni su dati non visti.

2. **Impostazione dei Callback per un Addestramento Robusto**

   * **ModelCheckpoint (Salvataggio del Modello Migliore):** Monitora la val_loss (errore di ricostruzione sul set di validazione). Il suo compito è salvare il modello solo quando si verifica un miglioramento della performance. Questo garantisce che, anche se l'addestramento continua, si mantenga sempre la versione migliore del modello in lstm_autoencoder_best.keras.

   * **EarlyStopping (Interruzione Anticipata):** Agisce come un meccanismo di sicurezza. Monitora anch'esso la val_loss e, se non rileva alcun miglioramento per un certo numero di epoche (patience=5), interrompe l'addestramento in anticipo. Questo è cruciale per prevenire l'overfitting (quando il modello impara troppo a memoria i dati di training) e per risparmiare tempo e risorse di calcolo.

   * **CSVLogger:** Registra i dettagli dell'addestramento (come la loss e la validation loss di ogni epoca) in un file training_log.csv. Questo permette di analizzare le prestazioni e tracciare le curve di apprendimento anche dopo la fine del processo.

3. **Avvio del Processo di Addestramento (model_lstm.fit)**
   * **Dati:** Riceve i generatori di dati (train_gen e val_gen), che forniscono continuamente batch di dati normalizzati direttamente dalla memoria di massa, evitando problemi di RAM.

   * **Epoche:** Sebbene siano impostate 50 epoche, l'addestramento si affiderà all'EarlyStopping per fermarsi non appena la performance sul set di validazione smette di migliorare.

   * **Controllo:** Tutti i callbacks definiti sono attivati per supervisionare e gestire il processo.

In [ ]:

# E' stato effetuato un test per capire le performance su RTX 4050 e delineare un numero di steps per epoca adeguato per completare un epoca in circa 30 minuti.

STEPS_PER_EPOCH_CALCOLATI = 200000
VALIDATION_STEPS_CALCOLATI = 40000

print(f"Avvio Addestramento LSTM su GPU RTX 4050")
print(f"Stima durata epoca: ~32 minuti")
print(f"Batch Size: {BATCH_SIZE}")
print("-" * 40)


checkpoint = ModelCheckpoint(
    'lstm_autoencoder_best.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,                 # Se non migliora per 5 epoche si stoppa
    mode='min',
    verbose=1,
    restore_best_weights=True
)

csv_logger = tf.keras.callbacks.CSVLogger('training_log.csv', append=True)

try:
    history_lstm = model_lstm.fit(
        train_gen,
        steps_per_epoch=STEPS_PER_EPOCH_CALCOLATI,
        epochs=50,                                          #Si ferma prima se non migliora quindi abbiamo messo un numero alto
        validation_data=val_gen,
        validation_steps=VALIDATION_STEPS_CALCOLATI,
        callbacks=[checkpoint, early_stopping, csv_logger],
        verbose=1
    )
    print("\nAddestramento Completato")
    
except KeyboardInterrupt:
    print("\nAddestramento interrotto manualmente. Il modello migliore è salvo in 'lstm_autoencoder_best.keras'")